# Data Investigation

In [ ]:
options(jupyter.plot_mimetypes = "image/svg+xml")
library(tidyverse)

In [ ]:
round_by <- function(x, divisor) {
    round(x / divisor) * divisor
}

In [ ]:
training_data <- read_csv('train_processed.csv')

In [ ]:
training_data

In [ ]:
training_data %>%
    group_by(Sex) %>%
    summarize(Total_Survived = sum(Survived))

In [ ]:
training_data %>%
    mutate(Age = round_by(Age, 5)) %>%
    group_by(Age, Sex) %>%
    summarize(Total_Survived = sum(Survived)) %>%
    ggplot(aes(x = Age, y = Total_Survived, fill = Sex)) +
    geom_col()

In [ ]:
training_data %>%
    mutate(Age = round_by(Age, 5)) %>%
    ggplot(aes(x = Sex, y = Age)) +
    geom_violin()

In [ ]:
training_data %>%
    ggplot(aes(x = Family_Size, y = Pclass)) +
    geom_jitter()